In [1]:
import os
import tensorflow.keras as kr
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Input, concatenate 
from tensorflow.keras.layers import GlobalAveragePooling2D, AveragePooling2D, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.optimizers import SGD 
from glob import glob
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.8.0


In [2]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
IMAGE_SIZE = [224, 224]
train_path = 'D://FYP Project//Dataset//Train'
valid_path = 'D://FYP Project//Dataset//Validation'

In [4]:
print("Training Data:")
for expression in os.listdir("D://FYP Project//Dataset//Train"):
    print(str(len(os.listdir("D://FYP Project//Dataset//Train//"+expression))) + " " + expression + " images.")
print("\nValidation Data:")    
for expression in os.listdir("D://FYP Project//Dataset//Validation"):
    print(str(len(os.listdir("D://FYP Project//Dataset//Validation//"+expression))) + " " + expression + " images.")

Training Data:
70001 Fake images.
70001 Real images.

Validation Data:
19641 Fake images.
19787 Real images.


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
training_set = train_datagen.flow_from_directory('D://FYP Project//Dataset//Train',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

Found 140002 images belonging to 2 classes.


In [7]:
test_set = test_datagen.flow_from_directory('D://FYP Project//Dataset//Validation',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 39428 images belonging to 2 classes.


In [8]:
training_set.class_indices

{'Fake': 0, 'Real': 1}

In [9]:
folders = glob('D://FYP Project//Dataset//Train//*')

In [10]:
folders

['D://FYP Project//Dataset//Train\\Fake',
 'D://FYP Project//Dataset//Train\\Real']

In [11]:
kernel_init = kr.initializers.glorot_uniform()
bias_init = kr.initializers.Constant(value=0.2)

In [12]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, 
                      bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, 
                      bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, 
                      bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, 
                      bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, 
                      bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, 
                       bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [13]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', 
           kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(2, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(2, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(2, activation='softmax', name='output')(x)

In [14]:
model = Model(input_layer, [x, x1, x2], name='GoogLeNet')

In [15]:
model.summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv_1_7x7/2 (Conv2D)          (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pool_1_3x3/2 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv_1_7x7/2[0][0]']           
 )                                                                                        

 max_pooling2d_3 (MaxPooling2D)  (None, 14, 14, 512)  0          ['inception_4a[0][0]']           
                                                                                                  
 conv2d_19 (Conv2D)             (None, 14, 14, 160)  82080       ['inception_4a[0][0]']           
                                                                                                  
 conv2d_21 (Conv2D)             (None, 14, 14, 224)  226016      ['conv2d_20[0][0]']              
                                                                                                  
 conv2d_23 (Conv2D)             (None, 14, 14, 64)   38464       ['conv2d_22[0][0]']              
                                                                                                  
 conv2d_24 (Conv2D)             (None, 14, 14, 64)   32832       ['max_pooling2d_3[0][0]']        
                                                                                                  
 inception

 conv2d_46 (Conv2D)             (None, 7, 7, 320)    461120      ['conv2d_45[0][0]']              
                                                                                                  
 conv2d_48 (Conv2D)             (None, 7, 7, 128)    102528      ['conv2d_47[0][0]']              
                                                                                                  
 conv2d_49 (Conv2D)             (None, 7, 7, 128)    106624      ['max_pooling2d_7[0][0]']        
                                                                                                  
 inception_5a (Concatenate)     (None, 7, 7, 832)    0           ['conv2d_44[0][0]',              
                                                                  'conv2d_46[0][0]',              
                                                                  'conv2d_48[0][0]',              
                                                                  'conv2d_49[0][0]']              
          

In [16]:
sgd = SGD(lr=0.01, momentum=0.9, nesterov=False)

C:\Users\ABDULLAH\anaconda3\envs\tensorflowgpu\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [17]:
model.compile(
    loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], 
    optimizer=sgd, 
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    training_set, 
    validation_data=test_set, 
    epochs=50, 
    steps_per_epoch=80)

Epoch 1/50
80/80 [==============================] - 548s 7s/step - loss: 2.1978 - output_loss: 0.7619 - auxilliary_output_1_loss: 0.7161 - auxilliary_output_2_loss: 0.7199 - output_accuracy: 0.4961 - auxilliary_output_1_accuracy: 0.5078 - auxilliary_output_2_accuracy: 0.5066 - val_loss: 2.0847 - val_output_loss: 0.6933 - val_auxilliary_output_1_loss: 0.6952 - val_auxilliary_output_2_loss: 0.6961 - val_output_accuracy: 0.4981 - val_auxilliary_output_1_accuracy: 0.4981 - val_auxilliary_output_2_accuracy: 0.4981
Epoch 2/50
80/80 [==============================] - 470s 6s/step - loss: 2.0870 - output_loss: 0.6994 - auxilliary_output_1_loss: 0.6938 - auxilliary_output_2_loss: 0.6938 - output_accuracy: 0.5105 - auxilliary_output_1_accuracy: 0.5020 - auxilliary_output_2_accuracy: 0.5055 - val_loss: 2.0812 - val_output_loss: 0.6949 - val_auxilliary_output_1_loss: 0.6932 - val_auxilliary_output_2_loss: 0.6931 - val_output_accuracy: 0.4981 - val_auxilliary_output_1_accuracy: 0.5019 - val_auxilli